In [ ]:
from transformers import AutoTokenizer
import os
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [12]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")
print(classifier("I absolutely love this product!"))



[{'label': 'POSITIVE', 'score': 0.9998854398727417}]


In [26]:
import pandas as pd
pathway = "/Users/hamidabdul/Library/CloudStorage/OneDrive-Personal/Projects/NLP/Code/sentiment-analyser/data/Reviews.csv"
df = pd.read_csv(pathway).rename(columns={"Text":"text"})
sample = df.sample(1000, random_state=42).copy()
  

In [25]:
clf = pipeline(
    "sentiment-analysis",
    model="distilbert-base-uncased-finetuned-sst-2-english",
    tokenizer="distilbert-base-uncased-finetuned-sst-2-english",
    device=-1  # set to 0 if you have GPU
)

preds = clf(batch, truncation=True, padding=True) 

In [27]:
# Predict (make sure to truncate/pad)
preds = clf(sample["text"].tolist(), truncation=True, padding=True)
sample["pred_label"] = [p["label"] for p in preds]
sample["pred_score"] = [float(p["score"]) for p in preds]

In [28]:
output_pathway = "/Users/hamidabdul/Library/CloudStorage/OneDrive-Personal/Projects/NLP/Code/sentiment-analyser/results/reviews_sentiment_sample1k.csv"
sample.to_csv(output_pathway, index=False)
print("Saved → results/reviews_sentiment_sample1k.csv")

Saved → results/reviews_sentiment_sample1k.csv


In [29]:
print(sample["pred_label"].value_counts(normalize=True).round(3))
print(sample.sort_values("pred_score").head(5)[["text","pred_label","pred_score"]])         # least confident
print(sample.sort_values("pred_score", ascending=False).head(5)[["text","pred_label","pred_score"]])  # most confident


POSITIVE    0.688
NEGATIVE    0.312
Name: pred_label, dtype: float64
                                                     text pred_label  \
481393  I use jar foods when I am on the go or if I di...   NEGATIVE   
115627  Lipton Yellow Label is CTC (crush, tear, curl)...   POSITIVE   
492103  Waffle Crisp has been my favorite cereal since...   POSITIVE   
1533    Good product and decent shipping. However, the...   NEGATIVE   
145795  This is an excellent "everything but the kitch...   NEGATIVE   

        pred_score  
481393    0.500153  
115627    0.511091  
492103    0.517810  
1533      0.518081  
145795    0.535795  
                                                     text pred_label  \
544874  These gummy snacks are wonderful! They are in ...   POSITIVE   
563828  These cookies are delicious! So soft and flavo...   POSITIVE   
426110  Bought this for my daughter who loved flavored...   POSITIVE   
540558  Timothy's Midnight Magic coffee is the best K-...   POSITIVE   
304693  Exce

In [30]:
THRESH = 0.60
sample["uncertain"] = sample["pred_score"] < THRESH
sample["uncertain"].mean()  # share of low-confidence rows


0.021